In [1]:
import h5py
import numpy
import matplotlib.pyplot as plt
import matplotlib.colors as colors

In [ ]:
f = h5py.File('data_grackle.hdf5')
f.keys()

In [ ]:
Tdim = f['Tdim'].value
Ddim = f['Ddim'].value
den_arr = f['density'].value
temp_arr = f['temperature'].value
total_t = f['total_t'].value

t_arr = numpy.zeros((400,300))
for i in range(400):
    t_arr[i,:] = numpy.logspace(-2 , numpy.log10(total_t[i]),300)


time_taken = f['time_taken'].value


logmin_temp, logmax_temp = numpy.log10(temp_arr.min()), numpy.log10(temp_arr.max())
logmin_den , logmax_den  = numpy.log10(den_arr.min()) , numpy.log10(den_arr.max())

dlogtemp = ( logmax_temp - logmin_temp )/ Tdim /2.0
dlogden  = ( logmax_den  - logmin_den )/ Ddim / 2.0
temp_arr = numpy.logspace( logmin_temp - dlogtemp , logmax_temp + dlogtemp , Tdim + 1 )
den_arr  = numpy.logspace( logmin_den - dlogden, logmax_den + dlogden, Ddim + 1 )


plot_performance(Tdim, Ddim, temp_arr,den_arr, time_taken, 
                title='grackle_run_time', performance = False, total_t = total_t, log=False)    

In [ ]:
zeros = numpy.zeros((400,300))
conserv_dict = {'H': zeros, 
                'H2':zeros,
                'He':zeros, 
                'de':zeros}

for key in f.keys():
    ele_charge = key.split('_')
    if ele_charge[0] in ['H', 'He']:
        
        name = ele_charge[0]
        conserv_dict[name] += f[key].value
        try:
            charge = eval(ele_charge[1]) - 1
            conserv_dict['de'] += charge*f[key].value
        except:
            if ele_charge[1][0] == 'm':             
                charge = eval(ele_charge[1][1]) - 1
                conserv_dict['de'] += charge*f[key].value

In [ ]:
 conserv_dict['H']*1.00794 +  conserv_dict['H2']*2.01588

In [ ]:
conserv_dict['He'][300]

In [ ]:



plot_performance(Tdim, Ddim, temp_arr,den_arr, time_taken, 
                title='grackle_run_time', performance = False, total_t = total_t, log=False)    

In [ ]:
def plot_performance(Tdim, Ddim, temp_arr,den_arr, time_taken, title=None, performance=True, log=True, total_t = total_t):

    
    temp_2d, den_2d = numpy.meshgrid(temp_arr, den_arr)
    
    time_taken_mat = numpy.array(time_taken).reshape((Ddim,Tdim))
    
    if performance:
        fig, ax = plt.subplots(1,2, figsize = (16,8))
        
        pcm = ax[0].pcolor( temp_2d, den_2d, total_t_mat)
        fig.colorbar(pcm, ax=ax[0], extend='max')


        ax[0].set_xscale('log')
        ax[0].set_yscale('log')
        ax[0].set_xlabel('Temperature (K)')
        ax[0].set_ylabel('Density ($cm^{-3}$)')
        
        if log:
            pcm = ax[1].pcolor( temp_2d, den_2d, time_taken_mat,
                              norm=colors.LogNorm(vmin=time_taken_mat.min(), vmax=time_taken_mat.max()))
        else:
            pcm = ax[1].pcolor( temp_2d, den_2d, time_taken_mat)
        fig.colorbar(pcm, ax=ax[1], extend='max')

        ax[1].set_xscale('log')
        ax[1].set_yscale('log')
        ax[1].set_xlabel('Temperature (K)')
        ax[1].set_ylabel('Density ($cm^{-3}$)')
        if title != None:
            ax[0].set_title(title+' time evolved ')
            ax[1].set_title(title+' run time')

        plt.show()
    else:
        fig, ax = plt.subplots(1,1, figsize = (8,8))
        if log:
            pcm = ax.pcolor( temp_2d, den_2d, time_taken_mat,
                              norm=colors.LogNorm(vmin=time_taken_mat.min(), vmax=time_taken_mat.max()))
        else:        
            pcm = ax.pcolor( temp_2d, den_2d, time_taken_mat)
        fig.colorbar(pcm, ax=ax, extend='max')

        ax.set_xscale('log')
        ax.set_yscale('log')
        ax.set_xlabel('Temperature (K)')
        ax.set_ylabel('Density ($cm^{-3}$)')
        if title != None:
            ax.set_title(title)

        plt.show()